# Micrograd

This is a re-implementation of Micrograd for eductation purposes after watching [this](https://www.youtube.com/watch?v=VMj-3S1tku0&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=1) video.

### Notes
- We can represent mathematical functions as **computation graphs**. 
    - A computational graph is a directed acyclic graphs (DAG).
    - Each node represents some variable.
        - The leaf nodes *(a.k.a. source nodes)* represent the inputs to the function.
        - The root node(s) *(a.k.a. sink nodes)* represents the output(s).
        - The intermediate nodes represnt intermediate steps of the function.
    - Each edge represents some mathematical operation (e.g sum, product, exponentiation).
- **Backpropagation** is a method to compute the derivatives of parameters w.r.t. a functions output. To do this, we step backwards in the computation graph (using topoligical sort) and accumulate the gradients according to the **chain rule** from calculus.
- **Neural networks** are just mathematical functions where the output is a prediction. We can compose a neural network with a loss function to calculate the **loss**.
- To train a neural network we must compute the **derivatives of all the learnable parameters** w.r.t. the loss, this can be done with backpropagation.
- **Autograd** is a backpropagation engine used by PyTorch to build computational graphs and compute the derivatives of the leaf nodes w.r.t. the output (usually a loss function). It is optimized to work with tensors and take advantage of parallel computing (using GPUs).
- **Note:** Backpropagation is more general and can apply to any mathematical function. It just happens to be very useful for training neural networks.

In this notebook, we build **micrograd**, a simplified version of autograd at the scalar level for eductation purposes. The API of the gradient engine and the neuron class is very similar to the PyTorch API.

### Extras
Most common deep-learning mistakes:
1. You didn't try to overfit a single batch first. 
2. You forgot to toggle train/eval mode for the net. 
3. You forgot to `.zero_grad()` (in pytorch) before `.backward()`. 
4. You passed softmaxed outputs to a loss that expects raw logits.
5. You didn't use `bias=False` for your Linear/Conv2d layer when using BatchNorm, or conversely forget to include it for the output layer.
6. Thinking `view()` and `permute()` are the same thing (& incorrectly using view)

[Defining new autograd functions in PyTorch](https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html#pytorch-defining-new-autograd-functions)

In [1]:
import math, random

### Gradient Engine

In [2]:
class Scalar:
    def __init__(self, value: float, children: set = set(), operation: str = "None"):
        self.value: float = value
        self.grad: float = 0.0
        self.children: set = children
        self.operation: str = operation
        self._backward: function = lambda: None

    def __repr__(self):
        return f"Scalar(value={self.value}, grad={self.grad})"

    def __str__(self):
        return f"\nScalar(value={self.value}, grad={self.grad}, children={self.children}, operation={self.operation})"

    def __add__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(self.value + other.value, set((self, other)), "+")

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward = _backward

        return out

    def __radd__(self, other):
        return self + other

    def __neg__(self):  # -self
        return self * -1

    def __sub__(self, other):  # self - other
        return self + (-other)

    def __mul__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(
            value=self.value * other.value, children=set((self, other)), operation="*"
        )

        # Product rule + Chain rule
        def _backward():
            self.grad += other.value * out.grad
            other.grad += self.value * out.grad

        out._backward = _backward

        return out

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(
            self.value / other.value, children=set((self, other)), operation="/"
        )

        # Quotient rule + Chain rule
        def _backward():
            self.grad += (1.0 / other.value) * out.grad
            other.grad += (-self.value / other.value**2) * out.grad

        out._backward = _backward

        return out

    def exp(self):
        out = Scalar(math.exp(self.value), children=set((self,)), operation="exp")

        def _backward():
            self.grad += out.value * out.grad

        out._backward = _backward

        return out

    def log(self):
        out = Scalar(math.log(self.value), children=set((self,)), operation="log")

        def _backward():
            self.grad += (1.0 / self.value) * out.grad

        out._backward = _backward

        return out

    def sigmoid(self):
        out = Scalar(
            1 / (1 + math.exp(-self.value)), children=set((self,)), operation="sigmoid"
        )

        def _backward():
            self.grad += out.value * (1.0 - out.value) * out.grad

        out._backward = _backward

        return out

    def backward(self):
        # Topologically sort the computational graph (DAG)
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v.children:
                    build_topo(child)
                topo.append(v)

        build_topo(self)

        # Backpropagate through the computational graph to accumilate gradients
        self.grad = 1.0
        for v in reversed(topo):
            v._backward()

### Gradient Engine Tests

In [3]:
x = Scalar(-4.0)
z = 2 * x + 2 + x
q = z.sigmoid() + z * x
h = (z * z).sigmoid()
y = h + q + q.log() / x.exp()
y.backward()
y.value, x.grad

(242.40610125372064, -253.43468221607841)

In [4]:
import torch

x = torch.Tensor([-4.0]).double()
x.requires_grad = True
z = 2 * x + 2 + x
q = z.sigmoid() + z * x
h = (z * z).sigmoid()
y = h + q + q.log() / x.exp()
y.backward()
y.data.item(), x.grad.item()

(242.40610125372064, -253.43468221607847)

### Neural Network

In [5]:
class Neuron:
    def __init__(self, inputs: int, activation: bool = True):
        self.inputs, self.activation = inputs, activation
        self.weights = [Scalar(random.uniform(-1, 1)) for _ in range(self.inputs)]
        self.bias = Scalar(0)

    def __call__(self, x: list[Scalar]):
        out = sum([w * xi for w, xi in zip(self.weights, x)]) + self.bias
        if self.activation:
            out = out.sigmoid()
        return out

    def parameters(self):
        out = self.weights + [self.bias]
        return out

    def __repr__(self):
        return f"Neuron(inputs={self.inputs}, activation={self.activation})"

In [6]:
# test
n = Neuron(5, activation=True)
x = [Scalar(1) for _ in range(5)]
n(x)

Scalar(value=0.4925157220764906, grad=0.0)

In [7]:
class Layer:
    def __init__(self, inputs: int, outputs: int, softmax: bool = False):
        self.inputs, self.outputs, self.softmax = inputs, outputs, softmax
        self.neurons = [
            Neuron(inputs, activation=(not softmax)) for _ in range(outputs)
        ]

    def __call__(self, x: list[Scalar]):
        out = [neuron(x) for neuron in self.neurons]
        if self.softmax:
            denom = sum([s.exp() for s in out])
            out = [s.exp() / denom for s in out]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        out = [param for neuron in self.neurons for param in neuron.parameters()]
        return out

    def __repr__(self):
        return f"Layer(inputs={self.inputs}, outputs={self.outputs}, softmax={self.softmax})"

In [8]:
# test
L = Layer(5, 3, softmax=True)
x = [Scalar(1) for _ in range(10)]
print(f"Sum: {sum([v.value for v in L(x)])}")
L(x)

Sum: 1.0


[Scalar(value=0.3419307179578387, grad=0.0),
 Scalar(value=0.08672889872577426, grad=0.0),
 Scalar(value=0.5713403833163871, grad=0.0)]

In [9]:
class MLP:
    def __init__(self, structure: list[int], final_layer_softmax=True):
        self.structure = structure
        assert len(structure) >= 2
        self.layers = [
            Layer(inputs, outputs, softmax=False)
            for inputs, outputs in zip(structure[:-2], structure[1:])
        ]
        self.layers += [
            Layer(structure[-2], structure[-1], softmax=final_layer_softmax)
        ]

    def __call__(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        return out

    def parameters(self):
        out = [param for layer in self.layers for param in layer.parameters()]
        return out

    def __repr__(self):
        return f"MLP(structure={self.structure})"

In [10]:
# test
mlp = MLP([5, 10, 10, 3])
x = [Scalar(1) for _ in range(5)]
print(f"Sum: {sum([v.value for v in mlp(x)])}")
mlp(x)

Sum: 1.0


[Scalar(value=0.3969758907525013, grad=0.0),
 Scalar(value=0.19735471920930067, grad=0.0),
 Scalar(value=0.4056693900381981, grad=0.0)]

### Training Example

In [11]:
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

In [ ]:
data = load_iris(return_X_y=True)

X = [[Scalar(x) for x in row] for row in data[0]]
print(f"X.shape: ({len(X)}, {len(X[0])})")


# Helper function
def label_to_list(label: int, num_labels: int) -> list[Scalar]:
    out = [0 for _ in range(num_labels)]
    out[label] = 1
    out = [Scalar(x) for x in out]
    return out


Y = [label_to_list(label, 3) for label in data[1]]
print(f"Y.shape: ({len(Y)}, {len(Y[0])})")

In [ ]:
mlp = MLP([4, 6, 6, 3], final_layer_softmax=True)

for epoch in range(1000):
    
    # forward pass
    preds = [mlp(x) for x in X]
    loss = -sum(
        [sum([p.log() * t for p, t in zip(pred, targ)]) for pred, targ in zip(preds, Y)]
    )

    # backward pass
    for p in mlp.parameters():
        p.grad = 0.0
    loss.backward()

    # update
    for p in mlp.parameters():
        p.value += -0.002 * p.grad

    print(f"{epoch}\t{loss.value}")

In [ ]:
def argmax(x):
    return max(range(len(x)), key=lambda i: x[i])


predictions = [argmax([s.value for s in row]) for row in [mlp(x) for x in X]]
actual = data[1]

print(f"Accuracy: {sum(predictions==actual)/len(actual)*100:.1f}%")